In [1]:
import pandas as pd
import gspread
from Query import Query
from Acesso import Login
from Email import Email
from Moeda import Moeda

CODE='19ncwHi1IG0dddhm_vnUgmUrhqKAQngPzs-dPk04eYJs'

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

querys={

    'Vendedor':

    """
    
    SELECT * FROM netfeira.vw_vendedor
    
    """,

    'Supervisor':

    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """,

    'Cliente':

    """

    SELECT * FROM netfeira.vw_cliente    
    
    """

}

def Main():

    gc = gspread.service_account(filename='.\JSON\keys.json')

    # Open a spreadsheet by title
    sh = gc.open_by_key(CODE)

    ws=sh.worksheet('LEADS')

    sheet=pd.DataFrame(ws.get_all_records())

    return sheet

    pass

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
sheet_df=Main()

In [4]:
sheet_df.columns

Index(['CNPJ', 'RAZÃO SOCIAL', 'CEP', 'NOME FANTASIA', 'ENDEREÇO', 'BAIRRO',
       'CIDADE', 'NUMERO', 'UF', 'PORTE', 'TELEFONE', 'EMAIL', 'VENDEDOR',
       'COMPROU', 'MOTIVO', 'DATA', 'OBSERVAÇÃO', 'ATENDIMENTO', 'FOTO'],
      dtype='object')

In [5]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [6]:
tabelas_df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro'],
      dtype='object')

In [7]:
tabelas_df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [8]:
tabelas_df['Vendedor']=tabelas_df['Vendedor'].merge(tabelas_df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro','ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente']]

In [9]:
tabelas_df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [10]:
tabelas_df['Vendedor']['ID Vendedor']=tabelas_df['Vendedor']['ID Vendedor'].str.strip()

In [11]:
sheet_df['VENDEDOR']=sheet_df['VENDEDOR'].str.strip()

In [12]:
sheet_df=sheet_df.merge(tabelas_df['Vendedor'],left_on='VENDEDOR',right_on='ID Vendedor',how='inner')[['CNPJ', 'RAZÃO SOCIAL', 'CEP', 'NOME FANTASIA', 'ENDEREÇO', 'BAIRRO',
       'CIDADE', 'NUMERO', 'UF', 'PORTE', 'TELEFONE', 'EMAIL', 'VENDEDOR','Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente',
       'COMPROU', 'MOTIVO', 'DATA', 'OBSERVAÇÃO', 'ATENDIMENTO', 'FOTO']]

In [13]:
sheet_df.loc[sheet_df['COMPROU']=='FALSE','COMPROU']='NÃO COMPROU'

In [14]:
sheet_df.loc[sheet_df['COMPROU']=='TRUE','COMPROU']='COMPROU'

In [15]:
sheet_df.loc[sheet_df['COMPROU']=='','COMPROU']='SEM ATENDIMENTO'

In [16]:
def FormatarCNPJ(cnpj):
    
    if(len(cnpj)>11 and len(cnpj)<14):
        
        cnpj=f'0{cnpj}'
        
        pass
    
    return cnpj
    
    pass

In [17]:
#formatar cnpj
sheet_df['CNPJ']=sheet_df['CNPJ'].astype('str')

sheet_df['CNPJ']=sheet_df['CNPJ'].apply(FormatarCNPJ)

In [18]:
sheet_df['Vendido']=sheet_df['CNPJ'].apply(lambda info: tabelas_df['Cliente']['CNPJ'][tabelas_df['Cliente']['CNPJ']==info].count())

In [19]:
sheet_df['Vendido']=sheet_df.apply(lambda info: 'SIM' if info['Vendido']>0 else 'NÃO',axis=1)

In [20]:
sheet_df[sheet_df['COMPROU']!='SEM ATENDIMENTO']

,CNPJ,RAZÃO SOCIAL,CEP,NOME FANTASIA,ENDEREÇO,BAIRRO,CIDADE,NUMERO,UF,PORTE,TELEFONE,EMAIL,VENDEDOR,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,ID Sup,Supervisor,Email Sup,ID Gerente,Gerente,Email Gerente,COMPROU,MOTIVO,DATA,OBSERVAÇÃO,ATENDIMENTO,FOTO,Vendido
80,43662839000106,K M SILVA PADARIA E RESTAURANTE,7500000,PAES E DOCES DONNA ROSA,CORONEL BERTOLDO,CENTRO,SANTA ISABEL,995,SP,-,(19)3873-1495,,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,NÃO É O SEGMENTO DO CLIENTE,,Cliente esta localizado no interior de SP. San...,FALSE,,NÃO
81,09622497000154,PAES E DOCES DALICE LTDA,7434100,PADARIA DALICE,ALBERTO HINOTO,JARDIM LIMOEIRO,ARUJA,2540,SP,Micro,(11)4652-1544,kalves@uol.com.br,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,NÃO É O SEGMENTO DO CLIENTE,,Telefone só chama,FALSE,,NÃO
82,18829951000133,MARCO ANTONIO VAC - HOTEL,7180270,HOTEL SANMARCO,SANTOS DUMONT,CUMBICA,GUARULHOS,554,SP,Pequena,55 (11) 2446-1061,mauriciostdaniel@gmail.com,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,AGENDAMENTO,14/07/2022,Mandei e-mail para o cliente,FALSE,,NÃO
83,10373089000195,PLAZA AVENIDA IPIRANGA HOTEL LTDA,1040000,-,IPIRANGA,REPUBLICA,SAO PAULO,1152,SP,Micro,55 (11) 3629-3600,diretoria@uniclasshotel.com.br,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,NÃO É O SEGMENTO DO CLIENTE,,Telefone não existe,FALSE,,NÃO
84,01771110000110,HOTEL FUAR LTDA,3064000,HOTEL FUAR,CELSO GARCIA,TATUAPE,SAO PAULO,5836,SP,Micro,(11)2092-3451,,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,AGENDAMENTO,14/07/2022,Retornar das 09:00 ás 14:00,FALSE,,NÃO
85,12955124000100,POUSADA & HOSTEL SAO PAULO LTDA,5409000,POUSADA & HOSTEL SAO PAULO,CAPOTE VALENTE,PINHEIROS,SAO PAULO,292,SP,Pequena,55 (11) 3865-1023,contato@contabilleonardis.com.br,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,NÃO É O SEGMENTO DO CLIENTE,,Telefone da contabilidade. Não passaram inform...,FALSE,,NÃO
86,01289049000179,HOTEL METROPOLIS LTDA,1215010,HOTEL METROPOLIS,HELVETIA,CAMPOS ELISEOS,SAO PAULO,973,SP,Micro,55 (11) 3331-7269,,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,NÃO É O SEGMENTO DO CLIENTE,,Telefone só ocupado,FALSE,,NÃO
87,62716733000184,HOSPEDARIA TERNURA LTDA,8115000,-,MARECHAL TITO,ITAIM PAULISTA,SAO PAULO,4400,SP,Micro,55 (11) 2567-9841,,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NÃO COMPROU,NÃO É O SEGMENTO DO CLIENTE,,Telefone não existe.,FALSE,,NÃO
88,05445962000178,ALESSANDRA CAMPOS MENDES,4079002,-,JUREMA,INDIANOPOLIS,SAO PAULO,925,SP,Micro,55 (11) 3284-4785,,RENATAAP,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,FERSOUZA,FERNANDA MEIRA,PEDIDOS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM

In [26]:
sheet_df.columns=sheet_df.columns.str.title()

In [29]:
sheet_df.rename(columns={'Vendedor':'ID Vendedor'},inplace=True)

In [34]:
grupo_df=sheet_df[['ID Vendedor','Nome Resumido','Equipe','Cnpj']].groupby(['ID Vendedor','Nome Resumido','Equipe'],as_index=False).count()

In [44]:
grupo_df['Atendimento']=grupo_df.apply(lambda info: len(sheet_df['Cnpj'].loc[(sheet_df['Comprou']!='SEM ATENDIMENTO')&(sheet_df['ID Vendedor']==info['ID Vendedor'])].unique().tolist()),axis=1)

In [47]:
grupo_df['Leads %']=grupo_df.apply(lambda info: round(info['Atendimento']/info['Cnpj'],4)*100,axis=1)

In [64]:
grupo_df['Conversão']=grupo_df.apply(lambda info: len(sheet_df['Cnpj'].loc[(sheet_df['Comprou']=='COMPROU')&(sheet_df['Vendido']=='SIM')&(sheet_df['ID Vendedor']==info['ID Vendedor'])].unique().tolist()),axis=1)

In [65]:
grupo_df['Convertido %']=grupo_df.apply(lambda info: round(info['Conversão']/info['Atendimento'],4)*100 if info['Atendimento']!=0 else 0,axis=1)

In [67]:
grupo_df.rename(columns={'Cnpj':'Leads'},inplace=True)

In [68]:
grupo_df

,ID Vendedor,Nome Resumido,Equipe,Leads,Atendimento,Leads %,Conversão,Convertido %
0,ALERUIZ,ALEXANDRE RUIZ,EQUIPE 6 (CLT),20,0,0.00,0,0.00
1,BRUNA,BRUNA LORENÇATO,EQUIPE 9 (ATIVO),200,12,6.00,0,0.00
2,CRISTIAN,CHRISTIAN OLIVEIRA,EQUIPE 6 (CLT),20,0,0.00,0,0.00
3,FONTOLAN,CELSO MOLINA,EQUIPE 3 (CLT),40,0,0.00,0,0.00
4,FRANCASS,FRANCISCO ROCHA,EQUIPE 6 (CLT),20,0,0.00,0,0.00
5,JUSCELIN,JUSCELINO JUNIOR,EQUIPE 6 (CLT),20,0,0.00,0,0.00
6,KOITI,KOITI ABE,EQUIPE 6 (CLT),20,0,0.00,0,0.00
7,LEOLOREN,LEONARDO LORENCETTI,EQUIPE 3 (CLT),20,1,5.00,0,0.00
8,MARCOSLI,MARCOS SILVA,EQUIPE 3 (CLT),30,0,0.00,0,0.00
9,MAYRAQS,MAYRA SANTOS,EQUIPE 9 (ATIVO),200,5,2.50,0,0.00
